# Project 1: Trends in General Happiness versus Financial Satisfaction and Work Satisfaction Over Time
## Group 11
- Elizabeth Hunter (mwm6nq)
- Michael Hijduk (aqt7bt)
- Eva Massarelli (ecm8yu)
- Anjali Mehta (wfn7ad)
- Anika Potluri (agu4yh)
## ! git clone https://www.github.com/DS3001/group11

# Summary
Research question(s), overview of methods and findings...

# Data
Describe the source of data

In [ ]:
import pandas as pd

df = pd.read_csv('./data/selectedData.csv')